# GDPR Chatbot Evaluation with LangSmith (Evaluate a RAG application)

## 🎯 What We're Doing

We're going to systematically test and evaluate your GDPR chatbot to make sure it's working properly. Think of this like giving your chatbot a "final exam" to see how well it performs!


In [2]:
from langsmith import Client

from langchain_openai import ChatOpenAI
from typing_extensions import Annotated, TypedDict

# Now import - this should work
from backend import ask_gdpr_question
print("✅ Successfully imported ask_gdpr_question!")

# print(f"✅ Project root: {project_root}")


client = Client()

/Users/guillermo/venvs/langchain_venv/lib/python3.11/site-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


✅ Environment setup from Streamlit secrets
✅ LangSmith tracing enabled from environment variables
✅ Successfully imported ask_gdpr_question!



---

## Step 1: Create Test Questions (The "Exam Paper")
Evaluation Dataset



In [ ]:
# Step 1: Create our test dataset - like writing exam questions
# We're creating real questions that users might ask, with correct answers
# This becomes our "quality control checklist"

# Create test dataset for GDPR chatbot
examples = [
    {
        "inputs": {"question": "How long can I store customer data?"},
        "outputs": {"answer": "Customer data can typically be stored for up to 10 years for tax and accounting purposes, but specific retention periods depend on legal requirements and business needs."},
    },
    {
        "inputs": {"question": "Do I need a data protection officer?"},
        "outputs": {"answer": "Small businesses typically don't need a dedicated data protection officer unless they process large amounts of sensitive data or engage in systematic monitoring."},
    },
    {
        "inputs": {"question": "What are the basic principles of data processing?"},
        "outputs": {"answer": "The basic principles include lawfulness, fairness, transparency, purpose limitation, data minimization, accuracy, storage limitation, integrity, and accountability."},
    },
    {
        "inputs": {"question": "Can I use surveillance cameras in my workshop?"},
        "outputs": {"answer": "Surveillance cameras are generally permitted for security purposes but must be proportionate, clearly marked, and not monitor private areas like bathrooms or break rooms."},
    },
    {
        "inputs": {"question": "What should I do in case of a data breach?"},
        "outputs": {"answer": "In case of a data breach, you should immediately document the incident, assess the risk, notify affected individuals if necessary, and report to the supervisory authority within 72 hours."},
    },
]

# Create dataset
dataset_name = "GDPR-Compliance-Assistant-Test"
dataset = client.create_dataset(dataset_name=dataset_name)
client.create_examples(
    dataset_id=dataset.id,
    examples=examples
)

print(f"✅ Created dataset: {dataset_name} with {len(examples)} test cases")

✅ Created dataset: GDPR-Compliance-Assistant-Test with 5 test cases


In [11]:
# Deutsche Testfragen mit kurzen, präzisen Antworten (<500 tokens)
examples_german = [
    {
        "inputs": {"question": "Wie lange darf ich Kundendaten speichern?"},
        "outputs": {"answer": "Kundendaten: Bis zu 10 Jahre für Steuerzwecke. Konkrete Fristen hängen von gesetzlichen Anforderungen ab."},
    },
    {
        "inputs": {"question": "Brauche ich einen Datenschutzbeauftragten?"},
        "outputs": {"answer": "Kleine Unternehmen meist nicht. Nur bei umfangreicher Datenverarbeitung oder systematischer Überwachung erforderlich."},
    },
    {
        "inputs": {"question": "Was sind die Grundsätze der Datenverarbeitung?"},
        "outputs": {"answer": "Rechtmäßigkeit, Zweckbindung, Datenminimierung, Richtigkeit, Speicherbegrenzung, Integrität und Rechenschaftspflicht."},
    },
    {
        "inputs": {"question": "Wie unterscheiden sich Aufbewahrungsfristen für Kunden- und Mitarbeiterdaten?"},
        "outputs": {"answer": "Kundendaten: Bis 10 Jahre (Steuer). Mitarbeiterdaten: Variabel (Lohnabrechnungen 6-10 Jahre, Bewerbungen 2-6 Monate)."},
    },
    {
        "inputs": {"question": "Was muss im Verzeichnis von Verarbeitungstätigkeiten stehen?"},
        "outputs": {"answer": "Verantwortlicher, Verarbeitungszwecke, Datenkategorien, Empfänger, Aufbewahrungsfristen, Sicherheitsmaßnahmen."},
    },
    {
        "inputs": {"question": "Kann ich personenbezogene Daten für KI-Training verwenden?"},
        "outputs": {"answer": "Nur mit spezifischer Einwilligung. Daten sollten anonymisiert werden. Folgenabschätzung oft erforderlich."},
    },
    {
        "inputs": {"question": "Was ist das Prinzip der Datenminimierung?"},
        "outputs": {"answer": "Nur notwendige Daten erheben und verarbeiten. Nicht mehr Daten als nötig für den Zweck sammeln."},
    },
]

# Create German dataset
dataset_name_german = "GDPR-Assistent-Deutsch-Kurz"
dataset_german = client.create_dataset(dataset_name=dataset_name_german)
client.create_examples(
    dataset_id=dataset_german.id,
    examples=examples_german
)

print(f"✅ Deutsche Testdaten erstellt: {len(examples_german)} Fragen")

# # Create German dataset
# dataset_name_german = "GDPR-Assistent-Deutsch-Test"
# dataset_german = client.create_dataset(dataset_name=dataset_name_german)
# client.create_examples(
#     dataset_id=dataset_german.id,
#     examples=examples_german
# )

✅ Deutsche Testdaten erstellt: 7 Fragen


In [ ]:
# # Deutsche Testfragen für den GDPR-Assistenten
# examples_german = [
#     {
#         "inputs": {"question": "Wie lange darf ich Kundendaten speichern?"},
#         "outputs": {"answer": "Kundendaten können in der Regel bis zu 10 Jahre für Steuer- und Buchführungszwecke gespeichert werden, aber die genauen Aufbewahrungsfristen hängen von den gesetzlichen Anforderungen und geschäftlichen Notwendigkeiten ab."},
#     },
#     {
#         "inputs": {"question": "Brauche ich einen Datenschutzbeauftragten?"},
#         "outputs": {"answer": "Kleine Unternehmen benötigen in der Regel keinen dedizierten Datenschutzbeauftragten, es sei denn, sie verarbeiten große Mengen sensibler Daten oder führen systematische Überwachungen durch."},
#     },
#     {
#         "inputs": {"question": "Was sind die Grundsätze der Datenverarbeitung?"},
#         "outputs": {"answer": "Die Grundsätze umfassen Rechtmäßigkeit, Treu und Glauben, Transparenz, Zweckbindung, Datenminimierung, Richtigkeit, Speicherbegrenzung, Integrität und Rechenschaftspflicht."},
#     },
#     {
#         "inputs": {"question": "Darf ich Überwachungskameras in meiner Werkstatt installieren?"},
#         "outputs": {"answer": "Überwachungskameras sind grundsätzlich zu Sicherheitszwecken erlaubt, müssen jedoch verhältnismäßig sein, klar gekennzeichnet werden und dürfen private Bereiche wie Toiletten oder Pausenräume nicht überwachen."},
#     },
#     {
#         "inputs": {"question": "Was muss ich bei einem Datenleck tun?"},
#         "outputs": {"answer": "Bei einem Datenleck sollten Sie den Vorfall sofort dokumentieren, das Risiko bewerten, betroffene Personen gegebenenfalls benachrichtigen und die Aufsichtsbehörde innerhalb von 72 Stunden informieren."},
#     },
#     {
#         "inputs": {"question": "Muss ich eine Einwilligung für Marketing-E-Mails einholen?"},
#         "outputs": {"answer": "Ja, für Marketing-E-Mails benötigen Sie eine aktive Einwilligung der betroffenen Personen. Vorausgefüllte Checkboxen oder Stilleschweigen gelten nicht als gültige Einwilligung."},
#     },
#     {
#         "inputs": {"question": "Was ist eine Datenschutz-Folgenabschätzung?"},
#         "outputs": {"answer": "Eine Datenschutz-Folgenabschätzung ist eine systematische Bewertung, die durchgeführt werden muss, wenn die Datenverarbeitung voraussichtlich ein hohes Risiko für die Rechte und Freiheiten von Personen birgt."},
#     },
#     {
#         "inputs": {"question": "Darf ich Fotos von meinen Handwerksarbeiten auf der Website zeigen?"},
#         "outputs": {"answer": "Ja, aber Sie müssen sicherstellen, dass keine personenbezogenen Daten wie Gesichter oder Kennzeichen sichtbar sind, oder eine Einwilligung der betroffenen Personen einholen."},
#     },
#     {
#         "inputs": {"question": "Wie behandle ich Beschwerden über Datenschutzverletzungen?"},
#         "outputs": {"answer": "Beschwerden über Datenschutzverletzungen müssen ernst genommen, dokumentiert und innerhalb eines Monats beantwortet werden. Bei berechtigten Beschwerden müssen Sie die Verarbeitung unverzüglich überprüfen."},
#     },
#     {
#         "inputs": {"question": "Was sind die Rechte der betroffenen Personen?"},
#         "outputs": {"answer": "Betroffene Personen haben das Recht auf Auskunft, Berichtigung, Löschung, Einschränkung der Verarbeitung, Datenübertragbarkeit und Widerspruch gegen die Verarbeitung."},
#     }
# ]

# # Create German dataset
# dataset_name_german = "GDPR-Assistent-Deutsch-Test"
# dataset_german = client.create_dataset(dataset_name=dataset_name_german)
# client.create_examples(
#     dataset_id=dataset_german.id,
#     examples=examples_german
# )

{'example_ids': ['39c2026b-5ee0-4c08-9a3f-cf245444bfea',
  '58acc9d7-24b1-4cd2-a7ec-d6c70b529137',
  '0ca41a1f-87bf-4850-ac82-ae59eba32a18',
  'faf60c26-09e2-4678-be75-03ce1fbfbdb4',
  '57334e57-7f25-49d7-90be-3037473be725',
  '009cd8de-a81d-4d3c-9013-5d811bf08624',
  '77e88859-0a90-4fd9-953b-78c12dc308b1',
  '159af693-9a5a-4d5b-ba83-01521f444504',
  'd983aa64-addd-40ec-9dad-899b0cb79df8',
  '06dac6bf-5c57-4d80-87d7-f4841ad642f9'],
 'count': 10}


**What this does:**
- Creates 5 realistic GDPR questions users might ask
- Provides the "correct" answers we expect
- Saves them in LangSmith as a permanent test set
- This is your chatbot's "final exam" that you can run anytime

**Why we do this:**
So we can test the chatbot consistently and track improvements over time!



---

## Step 2: Create Create Simple Evaluators
     Grading Rubrics (The "Answer Key")

- Create our evaluation criteria - like a teacher's grading rubric
- We're defining HOW to grade the chatbot's answers


In [12]:
# Simple relevance evaluator
class RelevanceGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    relevant: Annotated[bool, ..., "True if answer addresses the question"]

relevance_llm = ChatOpenAI(model="gpt-4o", temperature=0).with_structured_output(
    RelevanceGrade, method="json_schema"
)

def relevance(inputs: dict, outputs: dict) -> bool:
    """Check if answer is relevant to the question"""
    answer = f"QUESTION: {inputs['question']}\nANSWER: {outputs['answer']}"
    
    instructions = """You are evaluating a GDPR assistant. Check if the answer:
    1. Directly addresses the question
    2. Provides helpful information
    3. Stays on topic
    
    Return True if relevant, False if irrelevant."""
    
    grade = relevance_llm.invoke([
        {"role": "system", "content": instructions},
        {"role": "user", "content": answer}
    ])
    return grade["relevant"]

# Simple groundedness evaluator  
class GroundedGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning"]
    grounded: Annotated[bool, ..., "True if answer is based on documents"]

grounded_llm = ChatOpenAI(model="gpt-4o", temperature=0).with_structured_output(
    GroundedGrade, method="json_schema"
)

def groundedness(inputs: dict, outputs: dict) -> bool:
    """Check if answer is grounded in retrieved documents"""
    if not outputs.get("sources"):
        return False
    
    # Join source content
    source_text = "\n".join([s["content"][:500] for s in outputs["sources"][:3]])
    evaluation_text = f"QUESTION: {inputs['question']}\nANSWER: {outputs['answer']}\nSOURCES: {source_text}"
    
    instructions = """Check if the answer is supported by the source documents.
    Return True if the answer is grounded in the sources, False if it contains hallucinations."""
    
    grade = grounded_llm.invoke([
        {"role": "system", "content": instructions},
        {"role": "user", "content": evaluation_text}
    ])
    return grade["grounded"]

In [13]:
# Deutsche Bewertungskriterien
class GermanRelevanceGrade(TypedDict):
    erklärung: Annotated[str, ..., "Erkläre deine Bewertung"]
    relevant: Annotated[bool, ..., "True wenn die Antwort die Frage adressiert"]

german_relevance_llm = ChatOpenAI(model="gpt-4o", temperature=0).with_structured_output(
    GermanRelevanceGrade, method="json_schema"
)

def german_relevance(inputs: dict, outputs: dict) -> bool:
    """Prüft ob die Antwort relevant zur Frage ist (Deutsch)"""
    frage_antwort = f"FRAGE: {inputs['question']}\nANTWORT: {outputs['answer']}"
    
    anweisungen = """Du bewertest einen GDPR-Assistenten auf Deutsch. Prüfe ob die Antwort:
    1. Direkt auf die Frage eingeht
    2. Hilfreiche Informationen liefert  
    3. Zum Thema passt
    
    Gib True zurück wenn relevant, False wenn irrelevant."""
    
    bewertung = german_relevance_llm.invoke([
        {"role": "system", "content": anweisungen},
        {"role": "user", "content": frage_antwort}
    ])
    return bewertung["relevant"]


**What this does:**
- **Relevance Check**: Does the answer actually address the question?
- **Groundedness Check**: Is the answer based on your GDPR documents (not making things up)?

**Why we do this:**
So we have consistent, automated grading instead of guessing if answers are good!



----
## Step 3: Create Target Function 
      -> Connect to Your Chatbot (The "Student")

-  Connect our test system to your actual chatbot
-  This is like having the student sit down to take the exam


In [14]:
def target_function(inputs: dict) -> dict:
    """Wrapper function for evaluation"""
    question = inputs["question"]
    response = ask_gdpr_question(question, show_sources=True)
    return {
        "answer": response["answer"],
        "sources": response["sources"]
    }

**What this does:**
- Imports your real chatbot function
- Creates a wrapper that LangSmith can test
- Makes sure we're evaluating the exact same chatbot your users see

**Why we do this:**
To test the REAL system, not some demo version!


---

## Step 4: Run the Evaluation (The "Grading Session")

- Run the actual evaluation - like grading all the exam papers
- This automatically tests all questions and gives scores

In [15]:
# Run the evaluation
experiment_results = client.evaluate(
    target_function,
    data=dataset_name,
    evaluators=[relevance, groundedness],
    experiment_prefix="gdpr-assistant-v1",
    metadata={"version": "initial-release", "model": "gpt-3.5-turbo"}
)

print("✅ Evaluation completed!")
print(f"Check results at: https://smith.langchain.com")

View the evaluation results for experiment: 'gdpr-assistant-v1-d7660d09' at:
https://smith.langchain.com/o/d6a23e22-1e46-4329-9054-f145531ae12c/datasets/9a91ace1-74d7-4a70-8b79-700aab927304/compare?selectedSessions=e99cefe0-3b77-4e4b-aa7f-ab759f22d94e




0it [00:00, ?it/s]

KeyboardInterrupt: 

In [ ]:
# German evaluation run
experiment_results_german = client.evaluate(
    target_function,
    data=dataset_name_german,
    evaluators=[german_relevance, groundedness],
    experiment_prefix="GDPR-Assistent-Deutsch-Kurz",
    metadata={"version": "initial-release", "sprache": "deutsch"}
)

print("✅ Deutsche Evaluation abgeschlossen!")

# Explore results locally as a dataframe if you have pandas installed


View the evaluation results for experiment: 'GDPR-Assistent-Deutsch-Kurz-c6d193b9' at:
https://smith.langchain.com/o/d6a23e22-1e46-4329-9054-f145531ae12c/datasets/028dbf7e-d30c-4014-9b37-9513fc204ed1/compare?selectedSessions=f29d56de-1d7e-4acc-8dcb-9c3224a1263d




0it [00:00, ?it/s]

✅ Deutsche Evaluation abgeschlossen!


NameError: name 'experiment_results' is not defined

In [17]:
experiment_results_german.to_pandas()

,inputs.question,outputs.answer,outputs.sources,error,reference.answer,feedback.german_relevance,feedback.groundedness,execution_time,example_id,id
0,Wie unterscheiden sich Aufbewahrungsfristen fü...,Die Aufbewahrungsfristen für Kunden- und Mitar...,[{'content': 'Ob und wann die Aufbewahrung von...,None,Kundendaten: Bis 10 Jahre (Steuer). Mitarbeite...,True,True,2.857186,07ca842b-4591-4fa1-878e-d5533b33eadb,132f91b8-0c3d-4490-8182-42c6e19b57ca
1,Was sind die Grundsätze der Datenverarbeitung?,Die Grundsätze der Datenverarbeitung sind Rech...,[{'content': 'Rechtliche Grundlagen Artikel ...,None,"Rechtmäßigkeit, Zweckbindung, Datenminimierung...",True,True,1.643391,333ee8ff-53db-4c13-ae5c-1f35ec64ab8d,7c190b77-575b-43e6-a1f1-1f0cb690da49
2,Was muss im Verzeichnis von Verarbeitungstätig...,Im Verzeichnis von Verarbeitungstätigkeiten mü...,[{'content': 'Erläuterungen zum Verarbeitungsv...,None,"Verantwortlicher, Verarbeitungszwecke, Datenka...",True,False,2.217410,3c1de20b-6a11-4e3f-96f2-0c0d6f3a7ae0,bc681f4b-937f-4167-b391-c6ff34192260
3,Wie lange darf ich Kundendaten speichern?,Sie dürfen Kundendaten in der Regel bis zum Ab...,[{'content': 'Gesetzliche Löschfristen In ...,None,Kundendaten: Bis zu 10 Jahre für Steuerzwecke....,True,False,5.628028,4c64b8c0-431c-47d1-97fd-079cdde684d6,71fba1b4-d858-4e6b-9dd5-20664151b4a1
4,Kann ich personenbezogene Daten für KI-Trainin...,"Ja, du kannst personenbezogene Daten für KI-Tr...",[{'content': 'Praxisleitfaden KI & Datenschutz...,None,Nur mit spezifischer Einwilligung. Daten sollt...,True,True,1.537441,7285c93f-12f4-4ac4-8548-f12500e5f046,ac1f23d6-b62e-4aa7-a463-8f0ecfce9c7d
5,Was ist das Prinzip der Datenminimierung?,"Das Prinzip der Datenminimierung besagt, dass ...",[{'content': 'Praxisleitfaden KI & Datenschutz...,None,Nur notwendige Daten erheben und verarbeiten. ...,True,True,1.558778,84bea00b-66c6-4b4e-ba94-2695953195e5,9a267ce0-0d83-472e-8e02-0f11a4d60b5e
6,Brauche ich einen Datenschutzbeauftragten?,Als Gesundheitshandwerker müssen Sie keinen Da...,[{'content': 'wenn sie die Gesundheitsdaten er...,None,Kleine Unternehmen meist nicht. Nur bei umfang...,False,True,1.150993,925c1da1-f52b-4ea7-956f-2b42672d332c,6dbeb683-0377-4b17-8384-58ebee51abc4


In [18]:
# Convert to DataFrame
results_df = experiment_results_german.to_pandas()

# Export to CSV
csv_filename = "german_experiment_results.csv"
results_df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"✅ Experiment results exported: {csv_filename}")
print(f"📊 Total evaluations: {len(results_df)}")
print("\n📋 Columns available:")
print(results_df.columns.tolist())

✅ Experiment results exported: german_experiment_results.csv
📊 Total evaluations: 7

📋 Columns available:
['inputs.question', 'outputs.answer', 'outputs.sources', 'error', 'reference.answer', 'feedback.german_relevance', 'feedback.groundedness', 'execution_time', 'example_id', 'id']



**What this does:**
- Runs all 5 test questions through your chatbot
- Automatically grades each answer using our rubrics
- Generates detailed reports and scores
- Shows you exactly where the chatbot struggles

**Why we do this:**
To get concrete data on performance instead of just guessing!


---

## Step 5: View Results (The "Report Card")

- Check your results online - like getting the graded papers back


After running, visit:
1. **LangSmith Dashboard** → **Datasets** → "GDPR-Compliance-Assistant-Test"
2. **Experiments** → "gdpr-assistant-v1" to see scores
3. **Traces** to see individual test runs


**What you'll see in LangSmith:**
- ✅ **Overall scores** (like 4/5 questions answered well)
- 🔍 **Detailed breakdown** of each question
- 📊 **Performance metrics** over time
- 🐛 **Specific problems** to fix

**Why we do this:**
So you know exactly what to improve in your chatbot!



## Quick Alternative: Basic Evaluation

If you want something simpler:


In [ ]:
qaa
# Simple evaluation without custom evaluators
from langsmith.evaluation import evaluate

def simple_target(inputs):
    response = ask_gdpr_question(inputs["question"])
    return {"output": response["answer"]}

# Run with built-in evaluators
experiment_results = evaluate(
    simple_target,
    data=dataset_name,
    evaluators=["cot_qa"],  # Built-in correctness evaluator
)

# Explore results locally as a dataframe if you have pandas installed
experiment_results.to_pandas()


## What You'll Evaluate:

- **✅ Relevance**: Does the answer address the question?
- **✅ Groundedness**: Is the answer based on your GDPR documents?
- **✅ Performance**: Response times and token usage
- **✅ Consistency**: Same questions get similar answers

## Key Benefits:

1. **Automated Testing**: Run evaluation anytime you update your system
2. **Quality Metrics**: Track performance over time
3. **Debugging**: See exactly where answers go wrong
4. **Comparison**: Test different models/prompts side-by-side


---

## 🚀 The Big Picture

This evaluation system lets you:
1. **Test before deployment** - Make sure it works well
2. **Track improvements** - See if changes make it better or worse  
3. **Find specific problems** - Know exactly what to fix
4. **Compare versions** - Test different models or prompts

It is a **quality control department** for your AI assistant! 🎯

------
# Debug

In [ ]:
# {
#         "inputs": {"question": "Darf ich Überwachungskameras in meiner Werkstatt installieren?"},
#         "outputs": {"answer": "Ja, für Sicherheit. Muss verhältnismäßig sein, gekennzeichnet werden und private Bereiche ausschließen."},
#     },
#     {
#         "inputs": {"question": "Was muss ich bei einem Datenleck tun?"},
#         "outputs": {"answer": "Dokumentieren, Risiko bewerten, Betroffene benachrichtigen, Aufsichtsbehörde innerhalb 72 Stunden informieren."},
#     },
#     {
#         "inputs": {"question": "Muss ich eine Einwilligung für Marketing-E-Mails einholen?"},
#         "outputs": {"answer": "Ja, aktive Einwilligung erforderlich. Keine vorausgefüllten Checkboxen. Opt-in notwendig."},
#     },
#     {
#         "inputs": {"question": "Was ist eine Datenschutz-Folgenabschätzung?"},
#         "outputs": {"answer": "Systematische Risikobewertung bei hohem Risiko für Personenrechte. Vor Verarbeitung durchführen."},
#     },
#     {
#         "inputs": {"question": "Darf ich Fotos von meinen Handwerksarbeiten auf der Website zeigen?"},
#         "outputs": {"answer": "Ja, wenn keine personenbezogenen Daten sichtbar sind. Bei Personen: Einwilligung erforderlich."},
#     },
#     {
#         "inputs": {"question": "Wie behandle ich Beschwerden über Datenschutzverletzungen?"},
#         "outputs": {"answer": "Ernst nehmen, dokumentieren, innerhalb eines Monats antworten. Bei Berechtigung: Verarbeitung überprüfen."},
#     },
#     {
#         "inputs": {"question": "Was sind die Rechte der betroffenen Personen?"},
#         "outputs": {"answer": "Auskunft, Berichtigung, Löschung, Einschränkung, Datenübertragbarkeit, Widerspruch."},
#     },
#     {
#         "inputs": {"question": "Wie unterscheiden sich Aufbewahrungsfristen für Kunden- und Mitarbeiterdaten?"},
#         "outputs": {"answer": "Kundendaten: Bis 10 Jahre (Steuer). Mitarbeiterdaten: Variabel (Lohnabrechnungen 6-10 Jahre, Bewerbungen 2-6 Monate)."},
#     },
#     {
#         "inputs": {"question": "Was muss im Verzeichnis von Verarbeitungstätigkeiten stehen?"},
#         "outputs": {"answer": "Verantwortlicher, Verarbeitungszwecke, Datenkategorien, Empfänger, Aufbewahrungsfristen, Sicherheitsmaßnahmen."},
#     },
#     {
#         "inputs": {"question": "Kann ich personenbezogene Daten für KI-Training verwenden?"},
#         "outputs": {"answer": "Nur mit spezifischer Einwilligung. Daten sollten anonymisiert werden. Folgenabschätzung oft erforderlich."},
#     },
#     {
#         "inputs": {"question": "Was ist das Prinzip der Datenminimierung?"},
#         "outputs": {"answer": "Nur notwendige Daten erheben und verarbeiten. Nicht mehr Daten als nötig für den Zweck sammeln."},
#     },
#     {
#         "inputs": {"question": "Muss ich Datenschutz bei jedem neuen Mitarbeiter schulen?"},
#         "outputs": {"answer": "Ja, regelmäßige Sensibilisierung und Schulung aller Mitarbeiter ist gesetzlich vorgeschrieben."},
#     }